In [3]:
from ConvergenceAnalysis import *


In [4]:
fd_data

In [5]:
def process_data(base_path):
    base_high = base_path.joinpath('high-high')
    base_low = base_path.joinpath('low-low')

    mols = fd_data['Molecule'].to_list()
    low_data = get_database_residuals_z(mols, base_low)
    high_data = get_database_residuals_z(mols, base_high)
    # add column to low_data and high_data indicating low or high called ground state
    low_data['Ground Protocol'] = 'Low'
    high_data['Ground Protocol'] = 'High'
    # combine dataframe
    data = pd.concat([low_data, high_data])

    # concatenate the protocol and ground state columns to make a new column
    data['Response Protocol'] = data['Protocol'].astype(str)
    # if protocol is 'High' or 'Low' then set it to 'H' or 'L' 
    data.loc[data['Response Protocol'] == 'High', 'Protocol'] = 'High'
    data.loc[data['Response Protocol'] == 'Medium', 'Protocol'] = 'Low'
    data.loc[data['Response Protocol'] == 'Guess', 'Protocol'] = 'Guess'
    return data



In [6]:
november_absolute_path = Path('/mnt/data/madness_data/november_absolute_kain/')

fd_compare2 = Path('/mnt/data/madness_data/fd_compare2/')
fd_compare_data = process_data(fd_compare2)

november_absolute_data = process_data(november_absolute_path)

paper_path = Path('/home/adrianhurtado/projects/writing/mra-tdhf-polarizability/Figures')



In [7]:
november_absolute_data

In [8]:
data = fd_compare_data
data = november_absolute_data

sns.set_context("paper", font_scale=1.50)
sns.set_style("darkgrid")
sns.set_palette("colorblind")
style = 'Molecule'
hue = 'Response Protocol'
hue_order = ['Low', 'High']
data= data.query('Protocol.isin(@hue_order)')
# colormap is colorblind staring with index 1
colors = sns.color_palette("colorblind", 3)
# grab second and third colors in list
colors = [colors[1], colors[2]]
# create palette from colors
palette = sns.color_palette(colors)
s = 200
x = 'dz'
x = 'Absolute Residual Z'
g = sns.relplot(x=x, y='Absolute Percent Error', data=data, style=style, hue=hue, s=s,hue_order=hue_order,
                col='Ground Protocol',palette=colors )
# set the subplot titles to Ground Protocol = H or M
g.set_titles(col_template="Ground Protocol : {col_name}")
# increase the font of the legend

for ax in g.axes.flat:
    #ax.plot([1e-6,1e-2],[1e-6,1e-2], ls='--', color='black')
    ax.set_xscale('log')
    ax.set_yscale('log')
    # write an annotation indicating the green line is .005
    # only on first left plot
    if ax == g.axes.flat[0]:
        ax.axhline(.02, ls='--', color='orange')
        ax.annotate('0.02%', xy=(1e-4, .02), xytext=(1e-4, .022))
    else:
        ax.axhline(5e-3, ls='--', color='green')
        ax.annotate('0.005%', xy=(1e-4, .005), xytext=(1e-4, .006))
        
# Draw a y=x line on each plot
for ax in g.axes.flat:
    if x=='dz':
        ax.set_xlabel(r'$\Delta \rho_{z}(x)$')
    else:
        ax.set_xlabel('||f(x)-x||')
    # y label is percent error of static polarizability
    ax.set_title('{}'.format(ax.get_title()))

g.fig.gca().invert_xaxis()
# Change the marker size in the legend to be larger and more legible
for lh in g.legend.legend_handles: 
    lh._sizes = [100]
# save a high res version
#g.savefig(paper_path.joinpath('mra_convergence.png'),dpi=600)


In [9]:
data=november_absolute_data
# first rename Ground Protocol  and Response Protocol to GP and RP
data.rename(columns={'Ground Protocol':'GP','Response Protocol':'RP'},inplace=True)

low_data=data.query('GP=="Low"').query('RP=="Low"')
high_data=data.query('GP=="High"').query('RP=="High"')


data=pd.concat([low_data,high_data])
data

In [12]:
# violin plot strip plot

sns.set_context("paper", font_scale=1.50)
sns.set_style("darkgrid")
sns.set_palette("colorblind")
style = 'Molecule'
hue = 'Molecule'
kind='strip'
jitter=False
height=3
aspect_ratio=1.0


with sns.plotting_context('paper', font_scale=1.00):
    facet_kws = {'margin_titles': True, 'despine': False}
    g = sns.catplot(x='Protocol', y='Absolute Percent Error', hue='Molecule',
                    data=data,
                    kind=kind, dodge=jitter, jitter=jitter,
                    height=height, aspect=aspect_ratio, sharey=True,
                    facet_kws=facet_kws, alpha=0.9,
                    )
    ax=g.axes.flat[0]
    ax.axhline(.02, ls='--', color='orange')
    ax.axhline(.005, ls='--', color='green')
    ax.set_yscale('log', )
    # adjust legend
    g._legend.set_title('System')


g.savefig(paper_path.joinpath('mra_convergence.png'),dpi=600)



In [109]:

with sns.plotting_context('paper', font_scale=1.00):
    facet_kws = {'margin_titles': True, 'despine': False}
    g = sns.catplot(x='Protocol', y='Percent Error', hue='Molecule',
                    data=data,
                    kind=kind, dodge=jitter, jitter=jitter,
                    height=height, aspect=aspect_ratio, sharey=True,
                    facet_kws=facet_kws, alpha=0.9,
                    )
    ax=g.axes.flat[0]
    ax.axhline(.05, ls='--', color='orange')
    ax.axhline(.005, ls='--', color='green')
    ax.axhline(-.05, ls='--', color='orange')
    ax.axhline(-.005, ls='--', color='green')
    #ax.set_yscale('log', )



In [67]:
data = fd_compare_data
data = november_absolute_data

sns.set_context("paper", font_scale=1.50)
sns.set_style("darkgrid")
sns.set_palette("colorblind")
style = 'Molecule'
hue = 'Response Protocol'
s = 200
x = 'dz'
g = sns.relplot(x=x, y='Percent Error', data=data, style=style, hue=hue, s=s,
                col='Ground Protocol')
# set the subplot titles to Ground Protocol = H or M
g.set_titles(col_template="Ground Protocol : {col_name}")
# increase the font of the legend

for ax in g.axes.flat:
    ax.set_xscale('log')
    ax.set_yscale('symlog', linthresh=1e-4)
    # write an annotation indicating the green line is .005
    # only on first left plot
    if ax == g.axes.flat[0]:
        ax.axhline(.05, ls='--', color='orange')
        ax.axhline(-.05, ls='--', color='orange')
        ax.annotate('0.02%', xy=(1e-4, .02), xytext=(1e-4, .022))
        ax.annotate('-0.02%', xy=(1e-4, -.02), xytext=(1e-4, -.022))
    else:
        ax.axhline(.005, ls='--', color='green')
        ax.axhline(-.005, ls='--', color='green')
        ax.annotate('0.005%', xy=(1e-4, .005), xytext=(1e-4, .006))
        ax.annotate('-0.005%', xy=(1e-4, -.005), xytext=(1e-4, -.006))

# Draw a y=x line on each plot
for ax in g.axes.flat:
    ax.set_xlabel(r'$\Delta \rho_{z}(x)$')
    # y label is percent error of static polarizability

g.fig.gca().invert_xaxis()
# Change the marker size in the legend to be larger and more legible
for lh in g.legend.legend_handles:
    lh._sizes = [100]


In [10]:
MadnessResponse('NaCl','hf','dipole',november_absolute_path.joinpath('low-low')).plot_residuals()

In [18]:
data.query('Protocol=="Medium" and Molecule=="NaCl"')
     
 

In [20]:


sns.set_context("paper", font_scale=1)
sns.set_style("whitegrid")
sns.set_palette("colorblind")
style = 'Molecule'
hue = 'Protocol'
s = 200
x = 'dz'
y = 'Absolute Residual Z'

g = sns.relplot(x=y, y=x, data=data, hue=hue,style=style, s=s,
                col='Ground Protocol')

for ax in g.axes.flat:
    ax.set_yscale('log')
    ax.set_xscale('log')
# Change the marker size in the legend to be larger and more legible
for lh in g.legend.legend_handles:
    lh._sizes = [100]
    
# draw a y=x line on each plot

for ax in g.axes.flat:
    ax.plot([1e-6,1e-2],[1e-6,1e-2], ls='--', color='black')
    ax.set_xlabel('||f(x)-x||')
    ax.set_ylabel(r'$\Delta \rho_{z}(x)$')
    ax.set_title('{}'.format(ax.get_title()))

g.fig.gca().invert_xaxis()



In [21]:

mols=data.Molecule.unique().tolist()


In [22]:
from DataAnalysisClass import *
single = ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ', 'aug-cc-pV5Z', 'aug-cc-pV6Z']
single_polarized = ['aug-cc-pCVDZ', 'aug-cc-pCVTZ', 'aug-cc-pCVQZ']
double = ['d-aug-cc-pVDZ', 'd-aug-cc-pVTZ', 'd-aug-cc-pVQZ', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z']
double_polarized = ['d-aug-cc-pCVDZ', 'd-aug-cc-pCVTZ', 'd-aug-cc-pCVQZ']
all_basis_sets = single + single_polarized+ double  + double_polarized


In [23]:
low_data=PolarizabilityData(mols,'hf','dipole',all_basis_sets,november_absolute_path.joinpath
('low-low'),overwrite=True)
high_data=PolarizabilityData(mols,'hf','dipole',all_basis_sets,november_absolute_path
                                .joinpath
('high-high'),overwrite=True)


In [25]:
low_data.save_dfs()
high_data.save_dfs()



In [26]:

low_z_data=create_z_data(low_data)
# create Protocol column
low_z_data['Protocol']='Medium'
high_z_data=create_z_data(high_data)
high_z_data['Protocol']='High'
z_data=pd.concat([low_z_data,high_z_data])
z_data





In [27]:

z_abs_error_plot=compare_z_to_basis_set(z_data,y='Absolute Percent Error')
#z_error_plot=compare_z_to_basis_set(z_data,y='Percent Error')
z_abs_error_plot.fig.savefig(paper_path.joinpath('z_abs_error.png'),dpi=600)


In [ ]:
z_data.query('basis=="MRA"').sort_values('molecule')
